In [1]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi
from image_analysis.preprocess import mask

from tqdm import tqdm

In [2]:
pixel_size = 8

In [3]:
folder_pretreated = r'example_pretreated_images'
df = pd.read_excel("example_images.xlsx")

In [4]:
df.head()

,Sample name,HCR,"Animal size, cm","Regeneration time, dpa","Exact animal size (snout to tail), cm"
0,hcr21_2_5cm_6dpa_noif,21,3,6,2.7
1,hcr21_2_5cm_6dpa_p_hist,21,3,6,2.7
2,hcr21_2_5cm_7dpa_noif,21,3,7,2.9


In [5]:
for pc in [50]:
    for morph in ['shh', 'fgf', 'dusp']:
        df[f'volume_{pc}pc_um3_{morph}']  = pd.NA

In [6]:
for i, file_name in tqdm( enumerate( df['Sample name'].values) ):
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    for pc in [50]:
        for morph in ['shh', 'fgf', 'dusp']:
            if f'{morph}{pc}_clip' in keys:
                if morph != 'shh':
                    df.loc[df['Sample name'] == file_name, f'volume_{pc}pc_um3_{morph}']  = np.sum(
                        pixel_size**3 * mask.few_largest_components( image[f'{morph}{pc}_clip'] > 0, 2 )
                    )
                else:
                    df.loc[df['Sample name'] == file_name, f'volume_{pc}pc_um3_{morph}']  = np.sum(
                        pixel_size**3 * mask.largest_component( image[f'{morph}{pc}_clip'] > 0 )
                    )

3it [00:00, 38.78it/s]


In [7]:
df.to_excel("results/volumes.xlsx", index=False)

In [8]:
df

,Sample name,HCR,"Animal size, cm","Regeneration time, dpa","Exact animal size (snout to tail), cm",volume_50pc_um3_shh,volume_50pc_um3_fgf,volume_50pc_um3_dusp
0,hcr21_2_5cm_6dpa_noif,21,3,6,2.7,2663424,3409920,14226432
1,hcr21_2_5cm_6dpa_p_hist,21,3,6,2.7,1657344,1355776,20955136
2,hcr21_2_5cm_7dpa_noif,21,3,7,2.9,1359872,1728000,27598848
